In [1]:
import tensorflow as tf
import keras
import os

2024-01-03 13:45:42.583997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-03 13:45:42.584023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-03 13:45:42.585180: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-03 13:45:42.591126: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 13:45:43.261866: W tensorflow/compiler/tf2

In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
class buildmodel():
    def residual_block(self, inputs, filter,block_num,reduce=False):
        res = inputs
        x = keras.layers.Conv2D(filter,(3, 3), strides = 2 if reduce else 1,activation = 'relu', padding = 'same',name = f'conv_{block_num}_1')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_2')(x)
        if reduce == True:
            res = keras.layers.Conv2D(filter,(1, 1), strides = 2, activation = 'relu', padding = 'same',name = f'conv_skip_{block_num-1}')(res)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num-1}')([x,res])
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_3')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_4')(x)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num}')([x,res])
        return res
    
    def resnet18(self, num_outputs=1):
        # Inputs
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = keras.layers.Conv2D(64,(7, 7),strides = 2, activation = 'relu', padding = 'same',name = 'conv2D')(inputs)
        x = keras.layers.MaxPooling2D(pool_size = (3,3),strides = 2 ,padding = 'same',name = 'max_pool_1')(x)
        # Block 1
        x = self.residual_block(x,64,1,False)
        # Block 2
        x = self.residual_block(x,128,2,True)
        # Block 3
        x = self.residual_block(x,256,3,True)
        # Block 4
        x = self.residual_block(x,512,4,True)
        # Ouputs
        x = keras.layers.GlobalAveragePooling2D(name = 'global_pooling')(x)
        if num_outputs==1:
            outputs = keras.layers.Dense(num_outputs,activation = 'sigmoid',name = 'outputs_sigmoid')(x)
        else:
            outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs_softmax')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'resnet18')
        return model
    def __init__(self, optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy() , metrics=keras.metrics.BinaryAccuracy()):
        self.model = self.resnet18()
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
    def compile(self, optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy() , metrics=keras.metrics.BinaryAccuracy()):
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
        self.model.compile(optimizer = self.optimizer, loss = self.loss, metrics = self.metrics)
    @tf.function
    def train_step(self,x,y):
        with tf.GradientTape() as tape:
            logits = self.model(x, training=True)
            loss_value = self.loss(y, logits)
        grads = tape.gradient(loss_value, self.model.trainable_weights)
        self.metrics.update_state(y, logits)
        return loss_value,grads
    @tf.function
    def optimize_model(self, grads):
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))

2024-01-03 13:45:43.996864: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 13:45:44.064368: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 13:45:44.064544: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_gen = img_gen.flow_from_directory('./data',class_mode="binary",target_size=(224,224))

Found 24959 images belonging to 2 classes.


In [6]:
model_build = buildmodel()

In [7]:
import time
epochs = 10
start_train_time = time.time() 
for epoch in range(epochs):
    start_epoch = time.time()
    print("\nStart of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_gen):
        y_batch_train = y_batch_train.reshape(-1,1)
        # print(step, x_batch_train.shape, y_batch_train.shape)
        loss_value,grads = model_build.train_step(x_batch_train,y_batch_train)
        model_build.optimize_model(grads)
        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 32))
        if step==len(train_gen)-1:
            break
    # Display metrics at the end of each epoch.
    train_acc = model_build.metrics.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))
    print("Training time over epoch:", time.time()-start_epoch)
    # Reset training metrics at the end of each epoch
    model_build.metrics.reset_states()
print("Total time:", time.time()-start_train_time)


Start of epoch 0
Training loss (for one batch) at step 0: 0.7284
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6837
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.6953
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.6938
Seen so far: 19232 samples
Training acc over epoch: 0.5223
Training time over epoch: 50.33810234069824

Start of epoch 1
Training loss (for one batch) at step 0: 0.6891
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6707
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.6482
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.7024
Seen so far: 19232 samples
Training acc over epoch: 0.5564
Training time over epoch: 44.74661326408386

Start of epoch 2
Training loss (for one batch) at step 0: 0.6535
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.7055
Seen so far: 6432 samples
Training loss (for one batch) at step 400:

In [5]:
model_build.compile()
model_build.model.fit(train_gen, epochs=10)

Epoch 1/10


2024-01-03 13:45:53.856029: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-03 13:45:53.948390: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-03 13:45:54.482461: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-03 13:45:55.351707: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f1d45446060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-03 13:45:55.351726: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2024-01-03 13:45:55.356597: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1704264355.435250  753146 device_compiler.

317/780 [===========>..................] - ETA: 26s - loss: 0.7771 - binary_accuracy: 0.5313

/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


780/780 [==============================] - 55s 60ms/step - loss: 0.7208 - binary_accuracy: 0.5462
Epoch 2/10
780/780 [==============================] - 45s 58ms/step - loss: 0.6737 - binary_accuracy: 0.5785
Epoch 3/10
780/780 [==============================] - 45s 58ms/step - loss: 0.6586 - binary_accuracy: 0.6073
Epoch 4/10
780/780 [==============================] - 45s 58ms/step - loss: 0.6153 - binary_accuracy: 0.6613
Epoch 5/10
780/780 [==============================] - 45s 58ms/step - loss: 0.5775 - binary_accuracy: 0.6990
Epoch 6/10
780/780 [==============================] - 45s 58ms/step - loss: 0.5593 - binary_accuracy: 0.7121
Epoch 7/10
780/780 [==============================] - 45s 58ms/step - loss: 0.5156 - binary_accuracy: 0.7415
Epoch 8/10
780/780 [==============================] - 45s 58ms/step - loss: 0.4691 - binary_accuracy: 0.7752
Epoch 9/10
780/780 [==============================] - 45s 58ms/step - loss: 0.4070 - binary_accuracy: 0.8148
Epoch 10/10
780/780 [=========